# 税制政策分析の基礎

DSGEモデルを用いた税制政策の経済効果分析を学習します。

In [1]:
# 環境設定
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 共通インフラの読み込み
sys.path.append('notebooks')
from common import (
    setup_notebook_environment,
    load_baseline_model,
    create_research_simulator
)

# 環境初期化
project_root = setup_notebook_environment("02_tax_policy_fundamentals")

📁 プロジェクトルート: /Users/daisukeyoda/Documents/JapanTaxSimulator
📋 02_tax_policy_fundamentals 環境設定完了


In [2]:
# モデルとシミュレータの初期化
model = load_baseline_model()
simulator, status_info = create_research_simulator(model)

print(f"シミュレータ: {status_info['simulator_type']}")
print(f"研究整合性: {status_info['research_compliance']}")

✅ ベースラインモデル読み込み成功
🔄 定常状態を計算中...
✅ 定常状態計算完了
デバッグ: 利用可能フォント数 461
デバッグ: Hiraginoフォント: ['Hiragino Sans CNS', 'Hiragino Sans', 'Hiragino Mincho ProN']
デバッグ: フォント 'Hiragino Sans' を発見しました
✅ 日本語フォント使用: Hiragino Sans


Matplotlib is building the font cache; this may take a moment.


🔄 RESEARCH MODE: Development (warnings enabled)
🔄 RESEARCH MODE: Development (warnings enabled)
✅ Research-grade linearized DSGE model initialized
🎓 研究グレードシミュレータ初期化成功
シミュレータ: ResearchTaxSimulator
研究整合性: ✅ COMPLIANT


/Users/daisukeyoda/Documents/JapanTaxSimulator/src/models/simple_dsge.py:37: ResearchWarning: 🚨 SIMPLIFIED DSGE MODEL: This model uses only 8 variables and simplified equations. Economic assumptions differ significantly from full DSGE models. Results should NOT be used for research without empirical validation.
  warnings.warn(
/Users/daisukeyoda/Documents/JapanTaxSimulator/src/tax_simulator.py:43: ResearchWarning: Simplified DSGE model available. This uses different economic assumptions than the full model. Results may not be comparable.
  warnings.warn(


In [3]:
# 現在の税制確認
params = model.params
print("=== 現在の税制 ===")
print(f"消費税率: {params.tau_c:.1%}")
print(f"所得税率: {params.tau_l:.1%}")
print(f"法人税率: {params.tau_f:.1%}")

# 定常状態確認
ss = model.steady_state.to_dict()
print(f"\nGDP: {ss['Y']:.3f}")
print(f"消費: {ss['C']:.3f}")
print(f"投資: {ss['I']:.3f}")

=== 現在の税制 ===
消費税率: 10.0%
所得税率: 20.0%
法人税率: 30.0%

GDP: 0.363
消費: 0.266
投資: 0.019


In [10]:
# 消費税改革シミュレーション
from src.tax_simulator import TaxReform

consumption_tax_reform = TaxReform(
    name="消費税2%引き上げ",
    tau_c=0.12,  # 10% → 12%
    implementation='permanent'
)

print(f"改革案: {consumption_tax_reform.name}")
print(f"消費税率: {params.tau_c:.1%} → {consumption_tax_reform.tau_c:.1%}")

# シミュレーション実行
print("\n🔄 シミュレーション実行中...")
consumption_results = simulator.simulate_reform(
    reform=consumption_tax_reform,
    periods=20
)
print("✅ シミュレーション完了")

改革案: 消費税2%引き上げ
消費税率: 10.0% → 12.0%

🔄 シミュレーション実行中...
✅ シミュレーション完了


In [9]:
# 結果分析
if hasattr(consumption_results, 'steady_state_baseline') and hasattr(consumption_results, 'steady_state_reform'):
    baseline_ss = consumption_results.steady_state_baseline
    reform_ss = consumption_results.steady_state_reform
    
    print("=== 消費税引き上げの長期的影響 ===")
    
    variables = {'Y': 'GDP', 'C': '消費', 'I': '投資', 'L': '労働時間'}
    
    for var, jp_name in variables.items():
        if hasattr(baseline_ss, var) and hasattr(reform_ss, var):
            old_val = getattr(baseline_ss, var)
            new_val = getattr(reform_ss, var)
            if old_val != 0:
                change = (new_val - old_val) / old_val * 100
                direction = "📈" if change > 0 else "📉" if change < 0 else "➡️"
                print(f"{direction} {jp_name}: {change:+.2f}%")
    
    # 福利厚生への影響
    if hasattr(consumption_results, 'welfare_change'):
        welfare = consumption_results.welfare_change
        welfare_direction = "📈" if welfare > 0 else "📉" if welfare < 0 else "➡️"
        print(f"\n{welfare_direction} 家計福利: {welfare:+.2f}%")
else:
    print("⚠️ 詳細な比較データが利用できません")

=== 消費税引き上げの長期的影響 ===
📉 GDP: -4.62%
📉 消費: -2.55%
📉 投資: -33.75%
📉 労働時間: -3.69%

📉 家計福利: -1.09%


In [11]:
# 所得税改革シミュレーション
income_tax_reform = TaxReform(
    name="所得税3%引き下げ",
    tau_l=0.17,  # 20% → 17%
    implementation='permanent'
)

print(f"改革案: {income_tax_reform.name}")
print(f"所得税率: {params.tau_l:.1%} → {income_tax_reform.tau_l:.1%}")

# シミュレーション実行
print("\n🔄 シミュレーション実行中...")
income_results = simulator.simulate_reform(
    reform=income_tax_reform,
    periods=20
)
print("✅ シミュレーション完了")

改革案: 所得税3%引き下げ
所得税率: 20.0% → 17.0%

🔄 シミュレーション実行中...
✅ シミュレーション完了


In [7]:
# 所得税改革の結果分析
if hasattr(income_results, 'steady_state_baseline') and hasattr(income_results, 'steady_state_reform'):
    baseline_ss = income_results.steady_state_baseline
    reform_ss = income_results.steady_state_reform
    
    print("=== 所得税引き下げの長期的影響 ===")
    
    variables = {'Y': 'GDP', 'C': '消費', 'I': '投資', 'L': '労働時間'}
    
    for var, jp_name in variables.items():
        if hasattr(baseline_ss, var) and hasattr(reform_ss, var):
            old_val = getattr(baseline_ss, var)
            new_val = getattr(reform_ss, var)
            if old_val != 0:
                change = (new_val - old_val) / old_val * 100
                direction = "📈" if change > 0 else "📉" if change < 0 else "➡️"
                print(f"{direction} {jp_name}: {change:+.2f}%")
    
    # 福利厚生への影響
    if hasattr(income_results, 'welfare_change'):
        welfare = income_results.welfare_change
        welfare_direction = "📈" if welfare > 0 else "📉" if welfare < 0 else "➡️"
        print(f"\n{welfare_direction} 家計福利: {welfare:+.2f}%")

=== 所得税引き下げの長期的影響 ===
📈 GDP: +4.13%
📈 消費: +4.55%
📈 投資: +1.56%
📉 労働時間: -0.11%

📈 家計福利: +2.17%


## 政策提言

### 消費税政策
- 消費税引き上げは消費を抑制
- 税収確保には有効だが経済活動には負の影響

### 所得税政策  
- 所得税引き下げは労働供給を刺激
- 経済成長促進効果が期待される
- 税収減少とのトレードオフ

In [8]:
print("✅ 02_tax_policy_fundamentals 完了")
print("\n📋 学習内容:")
print("✅ 日本の税制構造理解")
print("✅ 消費税政策の経済効果分析")
print("✅ 所得税政策の経済効果分析")
print("✅ 政策提言の基礎")
print("\n🚀 次: 03_research_grade_simulation.ipynb")

✅ 02_tax_policy_fundamentals 完了

📋 学習内容:
✅ 日本の税制構造理解
✅ 消費税政策の経済効果分析
✅ 所得税政策の経済効果分析
✅ 政策提言の基礎

🚀 次: 03_research_grade_simulation.ipynb
